🌳 Random Forest 정의
- Random Forest는 여러 개의 결정 트리(Decision Tree)를 만들어 예측 결과를 앙상블(투표 or 평균) 하는 앙상블 학습 기법입니다.
- 분류(Classification) → 여러 트리의 다수결로 결정
- 회귀(Regression) → 여러 트리의 평균값으로 예측

🔧 Random Forest 메커니즘 <br>
🧠 핵심 아이디어: "여러 트리 → 더 나은 예측" <br>
| 단계                    | 설명                                                       |
| --------------------- | -------------------------------------------------------- |
| 1️⃣ **Bootstrap 샘플링** | 전체 데이터에서 **중복 허용하여 일부 샘플 무작위 추출** (각 트리마다 다르게)           |
| 2️⃣ **무작위 피처 선택**     | 트리의 각 분기점에서 **전체 피처 중 일부만 무작위로 선택**해 분할 기준 선택            |
| 3️⃣ **여러 트리 생성**      | 위 과정으로 **독립적인 트리들**을 생성                                  |
| 4️⃣ **앙상블 예측**        | 분류: 트리들의 예측 결과 **투표(Majority Voting)**<br>회귀: **예측값 평균** |


✅ Random Forest가 뛰어난 이유
| 이유               | 설명                                   |
| ---------------- | ------------------------------------ |
| 🎯 **과적합 방지**    | 여러 트리의 예측을 결합해 **개별 트리의 과적합 효과를 줄임** |
| 🔀 **다양성 확보**    | 각 트리마다 **다른 데이터 + 다른 피처** → 예측 다양성 ↑ |
| 📊 **피처 중요도 제공** | 각 피처의 분할 기여도를 합산하여 **중요도 출력 가능**     |


In [2]:
# 📦 라이브러리 import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, classification_report, make_scorer,
    precision_recall_curve, average_precision_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 📂 데이터 로드
df = pd.read_csv("../data/WA_Fn-UseC_-HR-Employee-Attrition.csv")
df['Attrition'] = LabelEncoder().fit_transform(df['Attrition'])


In [4]:
# 🎯 주요 13개 변수 선택
selected_features = [
    'Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
    'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'OverTime',
    'StockOptionLevel', 'TotalWorkingYears', 'YearsAtCompany',
    'YearsInCurrentRole', 'YearsWithCurrManager'
]

In [5]:
# 🔁 범주형 인코딩
df['OverTime'] = LabelEncoder().fit_transform(df['OverTime'])
df['MaritalStatus'] = LabelEncoder().fit_transform(df['MaritalStatus'])

In [6]:
# 🎯 X, y 분리
X = df[selected_features]
y = df['Attrition']

In [7]:
# ⚖️ 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [8]:
# 📈 SMOTE로 클래스 불균형 해결
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)


In [9]:
# 🔀 학습/테스트셋 분리
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)


In [10]:
# 🌳 RandomForest 모델 학습
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

In [11]:
# ✅ 교차검증 평가 지표 (5개)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorers = {
    'f1': make_scorer(f1_score),
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    #'roc_auc': make_scorer(roc_auc_score, needs_proba=True)
    'roc_auc': make_scorer(roc_auc_score)
}

cv_results = {metric: cross_val_score(model, X_train, y_train, cv=cv, scoring=scorer).mean()
              for metric, scorer in scorers.items()}

cv_results['f1(Test)'] = f1_score(y_test, y_pred)
# 📊 교차검증 + 테스트셋 성능 출력
print("\n📊 [Random Forest 교차검증 + 테스트셋 성능 지표]")
for metric, score in cv_results.items():
    print(f"{metric:10s}: {score:.4f}")




📊 [Random Forest 교차검증 + 테스트셋 성능 지표]
f1        : 0.8284
accuracy  : 0.8316
precision : 0.8442
recall    : 0.8134
roc_auc   : 0.8316
f1(Test)  : 0.8417


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# 🔧 하이퍼파라미터 후보 정의
param_grid = {
    'model__n_estimators': [100, 200],                  # 트리 개수
    'model__max_depth': [None, 5, 10],                  # 최대 깊이
    'model__min_samples_split': [2, 5],                 # 내부 노드 분할 최소 샘플 수
    'model__min_samples_leaf': [1, 2]                   # 리프 노드 최소 샘플 수
}

# ⚙️ 파이프라인 정의: SMOTE → RandomForest
pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),                           # 클래스 불균형 처리
    ('model', RandomForestClassifier(random_state=42))           # 랜덤 포레스트 모델
])

# 📊 평가지표 정의
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'accuracy': make_scorer(accuracy_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# 📂 결과 저장용 딕셔너리
results = {}

# 🔁 각각의 기준(f1, recall, precision)에 대해 하이퍼파라미터 튜닝 수행
for metric in ['precision', 'recall', 'f1']:
    grid = GridSearchCV(
        estimator=pipeline,                 # 파이프라인 전체를 그리드 서치 대상으로
        param_grid=param_grid,              # 랜덤 포레스트 하이퍼파라미터
        scoring=scoring,                    # 여러 평가지표
        refit=metric,                       # 해당 metric 기준으로 모델 선택
        cv=5,                               # 5-Fold 교차검증
        verbose=0,
        n_jobs=-1,
        return_train_score=True
    )

    # 🏋️ 하이퍼파라미터 탐색 실행
    grid.fit(X_train, y_train)

    # ✅ 최적 모델 정보 저장
    best_index = grid.best_index_
    results[metric] = {
        'best_params': grid.best_params_,
        'scores': {
            'Precision': grid.cv_results_['mean_test_precision'][best_index],
            'Recall': grid.cv_results_['mean_test_recall'][best_index],
            'F1 Score': grid.cv_results_['mean_test_f1'][best_index],
            'Accuracy': grid.cv_results_['mean_test_accuracy'][best_index],
            'ROC AUC': grid.cv_results_['mean_test_roc_auc'][best_index]
        }
    }

# 📤 결과 출력
for metric, content in results.items():
    print(f"\n🎯 [Refit 기준: {metric.upper()}]")
    print("Best Parameters:", content['best_params'])
    for score_name, val in content['scores'].items():
        print(f"{score_name:12}: {val:.4f}")



🎯 [Refit 기준: PRECISION]
Best Parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Precision   : 0.9242
Recall      : 0.9026
F1 Score    : 0.9132
Accuracy    : 0.9143
ROC AUC     : 0.9143

🎯 [Refit 기준: RECALL]
Best Parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5, 'model__n_estimators': 100}
Precision   : 0.9135
Recall      : 0.9077
F1 Score    : 0.9104
Accuracy    : 0.9107
ROC AUC     : 0.9108

🎯 [Refit 기준: F1]
Best Parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Precision   : 0.9216
Recall      : 0.9057
F1 Score    : 0.9134
Accuracy    : 0.9143
ROC AUC     : 0.9143


🔍 주요 하이퍼파라미터 설명
| 하이퍼파라미터             | 설명                            |
| ------------------- | ----------------------------- |
| `n_estimators`      | 사용하는 트리의 개수 (많을수록 성능 ↑, 시간 ↑) |
| `max_depth`         | 트리의 최대 깊이 제한                  |
| `min_samples_split` | 노드를 분할하기 위한 최소 샘플 수           |
| `min_samples_leaf`  | 리프 노드에 있어야 할 최소 샘플 수          |
